## Example of 3D Graph

Here we provide the example code for SphereNet on QM93D and MD17 datasets. You can easily replace SphereNet with SchNet and DimeNetPP by changing model name and model parameters.

In [1]:
import torch
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'../..')
from dig.threedgraph.dataset import QM93D
from dig.threedgraph.dataset import MD17
from dig.threedgraph.method import SphereNet #SchNet, DimeNetPP
from dig.threedgraph.method import run
from dig.threedgraph.evaluation import ThreeDEvaluator

In [2]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

### Example code for QM93D data
***Note***: '3D' means that the data includes positional information for atoms.

We trained a separate model for each target except for _gap_, which was predicted by taking _homo-lumo_. You can use default hyperparameters to get comparable results, we also tuned hyperparameters like lr, lr_decay_factor, lr_decay_step_size, cutoff, num_spherical, num_radial, basis_emb_size_dist, basis_emb_size_angle, basis_emb_size_torsion to achieve better performance. The values/search space for hyperparameters are listed in the Appendix of our paper.

The default hyperparameters for QM93D are:  
    &ensp; energy_and_force=False, cutoff=5.0, num_layers=4, hidden_channels=128, out_channels=1, int_emb_size=64,  
    &ensp; basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,  
    &ensp; num_spherical=7, num_radial=6, envelope_exponent=5,  
    &ensp; num_before_skip=1, num_after_skip=2, num_output_layers=3,  
    &ensp; epochs=500, batch_size=32, vt_batch_size=32, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=50.




#### Loading dataset

In [3]:
dataset = QM93D(root='dataset/')
target = 'U0'
dataset.data.y = dataset.data[target]

split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=110000, valid_size=10000, seed=42)

train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]
print('train, validaion, test:', len(train_dataset), len(valid_dataset), len(test_dataset))

train, validaion, test: 110000 10000 10831


#### Loading model, loss and evaluation function

The evaluation metric is mean absolute error (MAE).

In [4]:
model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4, 
        hidden_channels=128, out_channels=1, int_emb_size=64, 
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256, 
        num_spherical=7, num_radial=6, envelope_exponent=5, 
        num_before_skip=1, num_after_skip=2, num_output_layers=3, use_node_features=True
        )
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

#### Training

In [5]:
run3d = run()
run3d.run(device, train_dataset, valid_dataset, test_dataset, model, loss_func, evaluation, epochs=20, batch_size=32, vt_batch_size=32, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15)

#Params: 1898566

=====Epoch 1

Training...
100%|██████████| 3438/3438 [06:39<00:00,  8.61it/s]

Evaluating...

100%|██████████| 313/313 [00:21<00:00, 14.52it/s]

Testing...

100%|██████████| 339/339 [00:22<00:00, 14.97it/s]
{'Train': 0.7765793317604924, 'Validation': 0.829788088798523, 'Test': 0.8256096243858337}

=====Epoch 2

Training...

100%|██████████| 3438/3438 [06:32<00:00,  8.76it/s]

Evaluating...

100%|██████████| 313/313 [00:19<00:00, 16.32it/s]

Testing...

100%|██████████| 339/339 [00:20<00:00, 16.37it/s]
{'Train': 0.3513736664000606, 'Validation': 0.4852057099342346, 'Test': 0.4807925820350647}

=====Epoch 3

Training...

100%|██████████| 3438/3438 [06:32<00:00,  8.75it/s]

Evaluating...

100%|██████████| 313/313 [00:19<00:00, 16.47it/s]

Testing...

100%|██████████| 339/339 [00:20<00:00, 16.25it/s]
{'Train': 0.2866970902528785, 'Validation': 0.3925183415412903, 'Test': 0.3926387131214142}

=====Epoch 4

Training...

100%|██████████| 3438/3438 [06:29<00:00,  8.83it/s]

E


### Example code for MD17 data

We trained a separate model for each molecule. You can use default hyperparameters to get comparable results, we also tuned hyperparameters like lr, lr_decay_factor, lr_decay_step_size, batch_size, basis_emb_size_dist, basis_emb_size_angle, basis_emb_size_torsion to achieve better performance. The values/search space for hyperparameters are listed in the Appendix of our paper.

The default hyperparameters for MD17 are:  
    &ensp; energy_and_force=True, cutoff=5.0, num_layers=4, hidden_channels=128, out_channels=1, int_emb_size=64,  
    &ensp; basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,  
    &ensp; num_spherical=7, num_radial=6, envelope_exponent=5,  
    &ensp; num_before_skip=1, num_after_skip=2, num_output_layers=3,  
    &ensp; epochs=2000, batch_size=4, vt_batch_size=64, lr=0.001, lr_decay_factor=0.5, lr_decay_step_size=200.

#### Loading dataset

In [3]:
dataset_md17 = MD17(root='dataset/', name='aspirin')

split_idx_md17 = dataset_md17.get_idx_split(len(dataset_md17.data.y), train_size=1000, valid_size=10000, seed=42)

train_dataset_md17, valid_dataset_md17, test_dataset_md17 = dataset_md17[split_idx_md17['train']], dataset_md17[split_idx_md17['valid']], dataset_md17[split_idx_md17['test']]
print('train, validaion, test:', len(train_dataset_md17), len(valid_dataset_md17), len(test_dataset_md17))

train, validaion, test: 1000 10000 200762


#### Loading model, loss and evaluation function

We predict energy and take the negative of the derivative of the energy with respect to the atomic positions as predicted forces.

The evaluation metric is mean absolute error (MAE).

In [4]:
model_md17 = SphereNet(energy_and_force=True, cutoff=5.0, num_layers=4, 
        hidden_channels=128, out_channels=1, int_emb_size=64, 
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256, 
        num_spherical=7, num_radial=6, envelope_exponent=5, 
        num_before_skip=1, num_after_skip=2, num_output_layers=3 
        )
loss_func_md17 = torch.nn.L1Loss()
evaluation_md17 = ThreeDEvaluator()

#### Training

In [5]:
run3d_md17 = run()
run3d_md17.run(device, train_dataset_md17, valid_dataset_md17, test_dataset_md17, model_md17, loss_func_md17, evaluation_md17, epochs=5, batch_size=4, vt_batch_size=64, lr=0.001, lr_decay_factor=0.5, lr_decay_step_size=200, energy_and_force=True)

#Params: 1898566

=====Epoch 1

Training...


100%|██████████| 250/250 [01:50<00:00,  2.26it/s]



Evaluating...



100%|██████████| 157/157 [00:46<00:00,  3.37it/s]

{'Energy MAE': 10445.1953125, 'Force MAE': 196.6641845703125}


Testing...


...



Evaluating...



100%|██████████| 157/157 [00:46<00:00,  3.37it/s]

{'Energy MAE': 470.1788024902344, 'Force MAE': 105.1806640625}


Testing...



100%|██████████| 3137/3137 [15:25<00:00,  3.39it/s]

{'Energy MAE': 471.0114440917969, 'Force MAE': 105.5485610961914}

{'Train': 23443.38701171875, 'Validation': 10988.245208740234, 'Test': 11025.867553710938}
Best validation MAE so far: 10988.245208740234
Test MAE when got best validation result: 11025.867553710938
